# Exploring rMATS Output

rMATS calls and quantifies alternative splicing sites using RNA-seq data. Although most of the splicing sites are known sites from the GTF file rather than noval, we still want to know if any of the alternative events are noval from the GTF file. So here I used the rMATS output of the RNA-seq data from a sample using GENCODE v36. When looking for the splicing patterns, a match is called when the exon and its upstream and downstream exist. If the exon is found but either the upstream or downstream is missing, a match is not called from the transcript of the gene.

In [1]:
import pickle
from typing import List
from moPepGen import gtf
from moPepGen.parser import RMATSParser

with open('test/files/gencode_36_index/annotation.pickle', 'rb') as fh:
    annotation = pickle.load(fh)

## 1. Skipped Exon (SE)

In [2]:
path = '../rmats/output/CPT0208690010_SE.MATS.JC.txt'
skipped_exons = list(RMATSParser.parse(path, 'SE'))

In [11]:
results = []
i = 0
for record in skipped_exons:
    i += 1
    transcript_ids = annotation.genes[record.gene_id]
    transcripts = [annotation.transcripts[x] for x in transcript_ids.transcripts]
    retained = False
    skipped = False
    for transcript in transcripts:
        if retained and skipped:
            break
        it = iter(transcript.exon)
        exon = next(it, None)
        while exon:
            if int(exon.location.end) == record.upstream_exon_end:
                exon = next(it, None)
                
                if not exon:
                    continue

                if int(exon.location.start) == record.downstream_exon_start:
                    skipped = True
                    break
                if exon.location.start == record.exon_start and \
                        exon.location.end == record.exon_end:
                    exon = next(it, None)
                    if not exon:
                        continue
                    if int(exon.location.start) == record.downstream_exon_start:
                        retained = True
                        break
            exon = next(it, None)
    results.append((retained, skipped))

In [12]:
both, neither, skipped_only, retained_only = 0, 0, 0, 0
for skipped, retained in results:
    if skipped and retained:
        both += 1
    elif skipped:
        skipped_only += 1
    elif retained:
        retained_only += 1
    else:
        neither += 1

In [13]:
print(f'both:          {both}')
print(f'skipped only:  {skipped_only}')
print(f'retained only: {retained_only}')
print(f'neither:       {neither}')

both:          53133
skipped only:  11206
retained only: 12620
neither:       3698


+ 53133 genes have transcripts with or without the exon of interest. For those genes, no non-canonical peptides are resulted.
+ 11206 genes only have the transcripts that the exon of interest is skipped. For those genes, retaining the exon will generate non-canonical peptides.
+ 12620 genes only have the transcripts that teh exon of interest is retained. For those genes, the skipping the exon will generate non-canonical peptides.
+ 3698 genes have neither the skipped or retained transcript. This maybe because the upstream or downstream exon is also skipped. But since rMATS is not able to detect the skipping of multiple exons, we are then not able to know what really happened.

## 2. Alternative 5' Splicing Site (A5SS)

In [14]:
path = '../rmats/output/CPT0208690010_A5SS.MATS.JC.txt'
a5ss = list(RMATSParser.parse(path, 'A5SS'))

In [15]:
results = []
for record in a5ss:
    transcript_ids = annotation.genes[record.gene_id]
    transcripts = [annotation.transcripts[x] for x in transcript_ids.transcripts]
    short = False
    long = False
    for transcript in transcripts:
        if short and long:
            break
        if transcript.transcript.strand == 1:
            it = iter(transcript.exon)
        else:
            it = reversed(transcript.exon)
        exon = next(it, None)
        while exon:
            if int(exon.location.start) == record.long_exon_start and \
                    int(exon.location.end) == record.long_exon_end:
                exon = next(it, None)
                if exon and exon.location.start == record.flanking_exon_start:
                    long = True
                break
            if int(exon.location.start) == record.short_exon_start and \
                    int(exon.location.end) == record.short_exon_end:
                exon = next(it, None)
                if exon and exon.location.start == record.flanking_exon_start:
                    short = True
                break
            exon = next(it, None)
    results.append((long, short))

In [16]:
both, neither, long_only, short_only = 0, 0, 0, 0
for long, short in results:
    if long and short:
        both += 1
    elif long:
        long_only += 1
    elif short:
        short_only += 1
    else:
        neither += 1

In [17]:
print(f'both:       {both}')
print(f'long only:  {long_only}')
print(f'short only: {short_only}')
print(f'neither:    {neither}')

both:       7694
long only:  3666
short only: 4309
neither:    1653


+ 7694 genes have transcripts with both the long and short version of the exon. For those genes, no non-canonical peptides are resulted.
+ 3666 genes only have the long exon, so the A5SS will genereate non-caninical peptides.
+ 4309 genes only have the short exon. The A5SS also generates non-canonical peptides.
+ 1653 genes don't have the exon annotated. The alternative splicing is more complicated and can not be infered at this stage.

## 3. Alternative 3' Splicing Site (A3SS)

In [18]:
path = '../rmats/output/CPT0208690010_A3SS.MATS.JC.txt'
a3ss = list(RMATSParser.parse(path, 'A3SS'))

In [19]:
results = []
for record in a3ss:
    transcript_ids = annotation.genes[record.gene_id]
    transcripts = [annotation.transcripts[x] for x in transcript_ids.transcripts]
    short = False
    long = False
    for transcript in transcripts:
        if short and long:
            break
        if transcript.transcript.strand == 1:
            it = iter(transcript.exon)
        else:
            it = reversed(transcript.exon)
        exon = next(it, None)
        while exon:
            if int(exon.location.start) == record.flanking_exon_start:
                exon = next(it, None)
                if not exon:
                    break
                if exon.location.start == record.long_exon_start and \
                    int(exon.location.end) == record.long_exon_end:
                    long = True
                elif int(exon.location.start) == record.short_exon_start and \
                    int(exon.location.end) == record.short_exon_end:
                    short = True
                break
            exon = next(it, None)
    results.append((long, short))

In [20]:
both, neither, long_only, short_only = 0, 0, 0, 0
for long, short in results:
    if long and short:
        both += 1
    elif long:
        long_only += 1
    elif short:
        short_only += 1
    else:
        neither += 1

In [21]:
print(f'both:       {both}')
print(f'long only:  {long_only}')
print(f'short only: {short_only}')
print(f'neither:    {neither}')

both:       12134
long only:  6318
short only: 4272
neither:    1902


+ 12134 genes have transcripts with both the long and short version of the exon. For those genes, no non-canonical peptides are resulted.
+ 6318 genes only have the long exon, so the A5SS will genereate non-caninical peptides.
+ 4272 genes only have the short exon. The A5SS also generates non-canonical peptides.
+ 1902 genes don't have the exon annotated. The alternative splicing is more complicated and can not be infered at this stage.

## 4. Mutually Exclusive Exons (MXE)

In [22]:
path = '../rmats/output/CPT0208690010_MXE.MATS.JC.txt'
mxes = list(RMATSParser.parse(path, 'MXE'))

In [23]:
results = []
for record in mxes:
    transcript_ids = annotation.genes[record.gene_id]
    transcripts = [annotation.transcripts[x] for x in transcript_ids.transcripts]
    first, second, both = False, False, False
    for transcript in transcripts:
        if first and second:
            break
        it = iter(transcript.exon)
        exon = next(it, None)
        while exon:
            if int(exon.location.end) == record.upstream_exon_end:
                exon = next(it, None)
                if not exon:
                    break
                if int(exon.location.start) == record.first_exon_start and \
                        int(exon.location.end) == record.first_exon_end:
                    exon = next(it, None)
                    if not exon:
                        break
                    if int(exon.location.start) == record.downstream_exon_start:
                        first = True
                    elif int(exon.location.start) == record.second_exon_start and \
                            int(exon.location.end) == record.second_exon_end:
                        exon = next(it, None)
                        if not exon:
                            break
                        if int(exon.location.start) == record.downstream_exon_start:
                            both = True
                elif int(exon.location.start) == record.second_exon_start and \
                        int(exon.location.end) == record.second_exon_end:
                    exon = next(it, None)
                    if not exon:
                        break
                    if int(exon.location.start) == record.downstream_exon_start:
                        second = True
            exon = next(it, None)
    results.append((first, second, both))

In [24]:
both, neither, first_only, second_only = 0, 0, 0, 0
for i_first, i_second, i_both in results:
    if i_both:
        both += 1
    elif i_first:
        first_only += 1
    elif i_second:
        second_only += 1
    else:
        neither += 1

In [25]:
print(f'both:        {both}')
print(f'first only:  {first_only}')
print(f'second only: {second_only}')
print(f'neither:     {neither}')

both:        2691
first only:  3220
second only: 485
neither:     852


+ 2691 genes have both the first and second exons. For those genes, no non-canonical peptides are resulted.
+ 3220 genes only have the first exon, so the non-caninical peptides will be generated when the first exon is replaced with the second.
+ 485 genes only have the second exon. Non-caninical peptides will be generated when the second exon is replaced with the first.
+ 1902 genes don't have either version. It could also because the upstream or downstream exons are skipped. The alternative splicing is more complicated and can not be infered at this stage.

## 5. Retained Intron

In [3]:
path = '../rmats/output/CPT0208690010_RI.MATS.JC.txt'
ri = list(RMATSParser.parse(path, 'RI'))

In [9]:
results = []
for record in ri:
    tx_ids = annotation.genes[record.gene_id].transcripts
    tx_models:List[gtf.TranscriptAnnotationModel] = [annotation.transcripts[tx_id] for tx_id in tx_ids]
    spliced, retained = False, False
    for tx in tx_models:
        if spliced and retained:
            break
        it = iter(tx.exon)
        exon = next(it, None)
        while exon:
            if exon.location.start == record.retained_intron_exon_start:
                if exon.location.end == record.retained_intron_exon_end:
                    retained = True
                    exon = next(it, None)
                    continue
                if exon.location.end == record.upstream_exon_end:
                    exon = next(it, None)
                    if not exon:
                        break
                    if exon.location.start == record.downstream_exon_start and \
                            exon.location.end == record.downstream_exon_end:
                        spliced = True
            exon = next(it, None)
    results.append((spliced, retained))

In [12]:
spliced, retained, neither, both = 0, 0, 0, 0
for result in results:
    if result[0] and result[1]:
        both += 1
    elif result[0]:
        spliced += 1
    elif result[1]:
        retained += 1
    else:
        neither += 1

In [13]:
print(f'both:          {both}')
print(f'spliced only:  {spliced}')
print(f'retained only: {retained}')
print(f'neither:       {neither}')

both:          8818
spliced only:  0
retained only: 1303
neither:       3105


+ 8818 genes has transcripts with the intron both spliced and retained.
+ 0 genes only has transcripts with the intron spliced but not retained.
+ 1303 genes only have transcripts with the intron retained but not spliced.
+ 3105 genes don't have transcripts with either the intron retained or spliced.

An example of a gene that has only transcript with the target intron retained but not spliced. In this case below, the upstream and downstream exons have 5' or 3' alternative splicing sites. The upstream or downstream exon is also skipped in some transcripts. The start and end of the target intron is indeed valid splicing sites, but they just don't appear together in any of the reported transcript of the gene. So very likely, this is due to the incompleteness of annotation.

In [31]:
ri_retained = [i for i, result in enumerate(results) if not result[0] and result[1]]
print(f'ri_exon: {ri[ri_retained[0]].retained_intron_exon_start}-{ri[ri_retained[0]].retained_intron_exon_end}')
print(f'upstream: {ri[ri_retained[0]].upstream_exon_start}-{ri[ri_retained[0]].upstream_exon_end}')
print(f'downstream: {ri[ri_retained[0]].downstream_exon_start}-{ri[ri_retained[0]].downstream_exon_end}')

ri_exon: 122518838-122521428
upstream: 122518838-122519051
downstream: 122521384-122521428


In [32]:
[', '.join([f'{int(exon.location.start)}-{int(exon.location.end)}' for exon in annotation.transcripts[ex_id].exon]) for ex_id in annotation.genes[ri[ri_retained[0]].gene_id].transcripts]

['122503453-122505706, 122506833-122506923, 122508217-122508447, 122511108-122511188, 122515104-122515227, 122517226-122517430, 122518838-122519029, 122521384-122521428, 122522142-122522299, 122526847-122526936',
 '122505058-122505706, 122506833-122506923, 122508217-122508447, 122511108-122511188, 122515104-122515227, 122517226-122517430, 122518838-122519051, 122520503-122520581, 122521384-122521428, 122522142-122522299, 122526847-122526936',
 '122505058-122505706, 122506833-122506923, 122508217-122508450, 122511108-122511188, 122515104-122515227, 122517226-122517430, 122518838-122519029, 122520503-122520581, 122521384-122521428, 122522142-122522299, 122526847-122527000',
 '122505305-122505706, 122506833-122506923, 122508217-122508450, 122511108-122511188, 122515104-122515227, 122517226-122517430, 122518838-122519029, 122521295-122521428, 122522142-122522299, 122522798-122523756',
 '122505329-122505706, 122506833-122506923, 122508217-122508447, 122511108-122511188, 122513766-122513917'

This is an example of a gene that don't have the targt intron retained or spliced. Because the intron retaining even is defined strictly with the exact upstream and downstream exon start and end. In this gene below, the upstream and downstreams are either skipped have alternative splicing sites in each transcripts.

In [30]:
ri_neither = [i for i, result in enumerate(results) if not result[0] and not result[1]]
print(f'ri_exon: {ri[ri_neither[0]].retained_intron_exon_start}-{ri[ri_neither[0]].retained_intron_exon_end}')
print(f'upstream: {ri[ri_neither[0]].upstream_exon_start}-{ri[ri_neither[0]].upstream_exon_end}')
print(f'downstream: {ri[ri_neither[0]].downstream_exon_start}-{ri[ri_neither[0]].downstream_exon_end}')

ri_exon: 2612937-2613284
upstream: 2612937-2613055
downstream: 2613162-2613284


In [29]:
[', '.join([f'{int(exon.location.start)}-{int(exon.location.end)}' for exon in annotation.transcripts[ex_id].exon]) for ex_id in annotation.genes[ri[ri_neither[0]].gene_id].transcripts]

['2611477-2611901, 2612065-2612394',
 '2611477-2611901, 2612065-2612113, 2612220-2612812, 2612933-2613146',
 '2611477-2611901, 2612065-2612113, 2612220-2612253, 2612692-2612812, 2612933-2613209, 2613796-2614769',
 '2611477-2611901, 2612065-2612113, 2612220-2612271, 2612692-2612812, 2612933-2613209, 2613796-2614072, 2614317-2614587, 2614717-2614805',
 '2611477-2611901, 2612065-2612113, 2612692-2612812, 2612933-2613209, 2613796-2614072, 2614317-2614587, 2614717-2614805',
 '2611477-2611891, 2612692-2612812, 2612933-2613209, 2613796-2614072, 2614317-2614587, 2614717-2614812',
 '2611477-2611901, 2612065-2612113, 2612220-2612253, 2612692-2612812, 2612933-2613209, 2613796-2614055, 2614317-2614587, 2614717-2614824',
 '2611477-2611901, 2612065-2612113, 2612220-2612253, 2612692-2612812, 2612933-2613209, 2613796-2614072, 2614317-2614587, 2614717-2614849',
 '2612066-2612113, 2612220-2612253, 2612692-2612812, 2612933-2613209, 2613796-2614072, 2614317-2614417, 2614536-2614587, 2614717-2614790',
 '26

## Conclusion

rMATS is able to call 5 alternative splicing events. They are skipped exons (SE), alternative 5' splicing site, alternative 3' splicing site, mutual exclusive exons and retained intron. Although all alternative slicing sites come from the provided genomic annotation (GTF), not all of the events are included in it. For example for skipped exon, in the example above, there are 14,893 events of which only the exon skipped version is annotated, 4,765 events of which only the exon retained version is annotated. 17,672 events that have both the skipped and retained version, while 43,327 of them have neither skipped or retained version of transcript.